In [ ]:
# STEP 0: Install required packages and fonts
print("📦 Installing required packages...")
!pip install -q gdown matplotlib pillow pygments

print("📦 Installing fonts...")
!apt-get update -q
!apt-get install -y fonts-dejavu

import os
import sys
import shutil
import zipfile
from datetime import datetime
import gdown
import matplotlib.pyplot as plt
from PIL import Image
from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import ImageFormatter
import ast
import tokenize
import io

# Function to clean code: remove comments and blank lines
def clean_code(code):
    tokens = list(tokenize.tokenize(io.BytesIO(code.encode('utf-8')).readline))
    filtered_tokens = [token for token in tokens if token.type != tokenize.COMMENT]
    code_without_comments = tokenize.untokenize(filtered_tokens).decode('utf-8')
    lines = code_without_comments.splitlines()
    cleaned_lines = [line for line in lines if line.strip() != '']
    return '\n'.join(cleaned_lines)

# Function to check if the code is trivial (only def p(g): return g)
def is_trivial(tree):
    if len(tree.body) != 1:
        return False
    node = tree.body[0]
    if not isinstance(node, ast.FunctionDef):
        return False
    if node.name != 'p':
        return False
    if len(node.args.args) != 1 or node.args.args[0].arg != 'g':
        return False
    if len(node.body) != 1:
        return False
    stmt = node.body[0]
    if not isinstance(stmt, ast.Return):
        return False
    if not isinstance(stmt.value, ast.Name) or stmt.value.id != 'g':
        return False
    return True

# STEP 1: Banner
print("=" * 80)
print("🚀 GOOGLE DRIVE FOLDER DOWNLOADER & EXTRACTOR 🚀".center(80))
print("=" * 80)
print("📝 Credit: To daosyduyminh".center(80))
print("=" * 80)
print()

# STEP 2: Download Google Drive folder
url = "https://drive.google.com/drive/folders/1HLlj6YUHH4dCJ9Hu8pQDQ4fTCfsL86Yt"
print("📥 Downloading from Google Drive folder...")
downloaded_files = gdown.download_folder(url, quiet=False, use_cookies=False)
if not downloaded_files:
    print("❌ No files downloaded. Exiting.")
    sys.exit(1)
print(f"✅ Downloaded {len(downloaded_files)} files")

# STEP 3: Move and extract ZIP
src_path = "/kaggle/working/Golf Code/submission.zip"
dst_path = "/kaggle/working/submission.zip"

if os.path.exists(src_path):
    shutil.move(src_path, dst_path)
    print(f"✅ Moved ZIP to {dst_path}")
else:
    print("❌ submission.zip not found!")
    sys.exit(1)

extract_dir = "/kaggle/working/extracted"
os.makedirs(extract_dir, exist_ok=True)

print("📂 Extracting ZIP file...")
with zipfile.ZipFile(dst_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print("✅ Extraction complete!")

# STEP 4: Analyze extracted content
print("📊 Analyzing extracted files...")
all_files = os.listdir(extract_dir)
task_files = [f for f in all_files if f.startswith("task") and f.endswith(".py")]

print(f"🔢 Total files: {len(all_files)}")
print(f"🐍 Python task files: {len(task_files)}")

# STEP 5: Render non-trivial task files as code images
output_img_dir = "/kaggle/working/task_images"
os.makedirs(output_img_dir, exist_ok=True)

print("🖼️ Rendering non-trivial task files as code images (IDE style)...")

rendered_images = []
for task_file in sorted(task_files):
    task_path = os.path.join(extract_dir, task_file)
    try:
        with open(task_path, 'r', encoding='utf-8') as f:
            code = f.read()
    except Exception as e:
        print(f"⚠️ Skipping {task_file}: failed to read ({e})")
        continue

    # Clean the code by removing comments and blank lines
    cleaned_code = clean_code(code)

    # Check if the cleaned code is syntactically valid and trivial
    try:
        tree = ast.parse(cleaned_code)
    except SyntaxError as e:
        print(f"⚠️ Skipping {task_file}: syntax error in cleaned code ({e})")
        continue

    if is_trivial(tree):
        print(f"⚠️ Skipping {task_file}: contains only the trivial function 'def p(g): return g'")
        continue

    # Render the cleaned code as an image
    try:
        formatter = ImageFormatter(
            line_numbers=True,
            style='monokai',
            image_format='PNG',
            font_size=14
        )
        image_path = os.path.join(output_img_dir, task_file.replace('.py', '.png'))
        with open(image_path, 'wb') as img_file:
            img_file.write(highlight(cleaned_code, PythonLexer(), formatter))
        rendered_images.append(image_path)
    except Exception as e:
        print(f"⚠️ Skipping {task_file}: rendering failed ({e})")
        continue

print(f"✅ Rendered {len(rendered_images)} code files as images.")

# STEP 6: Display rendered images
print("🖼️ Displaying rendered code images...")
for img_path in rendered_images:
    try:
        print(f"📄 {os.path.basename(img_path)}")
        img = Image.open(img_path)
        plt.figure(figsize=(12, 6))
        plt.imshow(img)
        plt.axis('off')
        plt.title(os.path.basename(img_path))
        plt.show()
    except Exception as e:
        print(f"⚠️ Could not display {img_path}: {e}")

# FINAL SUMMARY
print("=" * 80)
print("🎉 ALL TASKS RENDERED SUCCESSFULLY 🎉".center(80))
print(f"🖼️ Total code images rendered: {len(rendered_images)}")
print(f"📅 Finished at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("=" * 80)